In [1]:
import os
import PyImbalReg as pir
import pandas as pd
import numpy as np 
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import RandomOverSampler
import seaborn as sns
import matplotlib.pyplot as plt
import resreg

In [2]:
df = pd.read_csv("/home/usr/Documents/reg-class-fault-detection/dataset/kpi_fuller_df.csv")

In [3]:
def label_func(faultD):
  if(faultD == 0): return 0
  elif(faultD < 500): return 1
  else: return 2

In [4]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def reg_acc(md, data, test_labels):
  test_predictions = md.predict(data).flatten()
  r2 = r2_score(test_labels, test_predictions)
  rmse = np.sqrt(mean_squared_error(test_labels, test_predictions))
  mae = mean_absolute_error(test_labels, test_predictions)

  print(f'r2 score: {r2}')
  print(f'rmse score: {rmse}')
  print(f'mae score: {mae}')
  # print(f'erro medio de: {rmse} segundos')

  plt.scatter(test_labels, test_predictions)
  plt.xlabel('True Values')
  plt.ylabel('Predictions')
  plt.axis('equal')
  plt.axis('square')
  plt.xlim([0,plt.xlim()[1]])
  plt.ylim([0,plt.ylim()[1]])
  _ = plt.plot([-100, 100], [-100, 100])

In [5]:
df = df.sample(frac=1)
test_data = df[:(len(df)//5)]
train_data = df[(len(df)//5):]

X_test = test_data.drop(columns=['Unnamed: 0','NE ID','endTime', 'relation','fault_duration']).values
y_test_fd = test_data.fault_duration.values
y_test = test_data.fault_duration.apply(label_func).values

train_data['fault'] = train_data['fault_duration'].apply(label_func)

train_data = train_data.dropna()
train_data = train_data.drop(columns=['Unnamed: 0','NE ID','endTime','relation'])

/tmp/ipykernel_3884/1550130980.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['fault'] = train_data['fault_duration'].apply(label_func)


In [6]:
def bin_func(faultD):
  if(faultD == 0): return 0
  else: return (faultD//500 + 1)

In [7]:
train_data['bin'] = train_data.fault_duration.apply(bin_func)

In [8]:
bin_freq = train_data.bin.value_counts().values
bin_keys = train_data.bin.value_counts().keys()
sum = np.sum(bin_freq)

relevance_values = [ (sum - b)/sum for b in bin_freq]

relevance_dict = list(zip(bin_keys, relevance_values))

In [9]:
relevance_dict.sort()

In [10]:
def relevance_function(sample):
    return relevance_dict[int(sample)][1]



In [11]:
y = train_data.fault_duration.values
X = train_data.drop(columns=['fault','fault_duration']).values

faults = train_data[train_data['fault_duration'] > 0]
X_fault = faults.drop(columns=['fault','fault_duration']).values
y_fault = faults.fault_duration.values

X_test = test_data.drop(columns=['Unnamed: 0','NE ID','endTime', 'relation','fault_duration']).values
y_test_fd = test_data.fault_duration.values
y_test = test_data.fault_duration.apply(label_func).values


In [12]:
relevance = train_data.bin.apply(relevance_function)

In [ ]:

X_train, y_train = resreg.smoter(X, y, relevance, relevance_threshold=0.5, k=5, over='balance', random_state=0)